<a href="https://colab.research.google.com/github/aswinaus/ML/blob/main/Document_Ingestion_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
┌──────────────────────────────┐
│     Azure Data Lake Gen2     │
│ ┌──────────────────────────┐ │
│ │  Millions of Documents   │ │
│ └──────────────────────────┘ │
└────────────┬─────────────────┘
             │
             ▼
┌──────────────────────────────┐
│   Azure Databricks Cluster   │
│   (Apache Spark + Python)    │
└────────────┬─────────────────┘
             │
             ▼
┌──────────────────────────────┐
│   Document Ingestion Layer   │
│ - Read from ADLS             │
│ - Use Spark DataFrame        │
└────────────┬─────────────────┘
             │
             ▼
┌──────────────────────────────┐
│     Data Cleansing Layer     │
│ - Detect PII with Regex/Spark NLP │
│ - Mask or remove sensitive data   │
└────────────┬─────────────────┘
             │
             ▼
┌──────────────────────────────┐
│     LLM Classification API   │
│ - Azure OpenAI / OpenAI GPT  │
│ - Prompt-based classification│
│ - Category: HR, Legal, etc.  │
└────────────┬─────────────────┘
             │
             ▼
┌──────────────────────────────┐
│   Classification Output      │
│ - Store back to ADLS         │
│ - Save in Delta Table        │
│ - Optionally send to SQL DB  │
└──────────────────────────────┘



[Azure Data Lake / Blob Storage]
          │
          ▼
[Azure Databricks Notebook]
  - Reads document URLs
  - Calls Document Intelligence API
  - Extracts structured text/data
          │
          ▼
[Data Cleansing / PII Redaction (Spark)]
          │
          ▼
[LLM Classification / Embedding]
          │
          ▼
[Save to Delta Lake / Cosmos DB / etc.]


                 ┌──────────────────────────────┐
                 │  SharePoint / Azure DataLake │
                 └────────────┬─────────────────┘
                              │
                  ┌──────────▼────────────┐
                  │ Document Intelligence |
                  | MultiLingual(OCR)     |
                  | Handwritten Recognition│
                  └──────────┬────────────┘
                             │
                      ┌──────▼──────┐
                      │ Spark (in   │
                      │ Databricks) │
                      └──────┬──────┘
         ┌───────────────────┴─────────────────────┐
         │ Clean + redact PII                      │
         │ LLM classification (batched)
         | Azure SQL                               │
         │ Human validation (HITL via PowerBI/UI)  │
         └───────────────────┬─────────────────────┘
                             │
                ┌────────────▼────────────┐
                │ Final redacted + tagged │
                │ docs ()  │
                └────────────┬────────────┘
                             │
         ┌───────────────────▼────────────────────┐
         │ Embedding (Azure OpenAI)    │
         └───────────────────┬────────────────────┘
                             │
                 ┌───────────▼────────────┐
                 │ Azure Cognitive Search │
                 └───────────┬────────────┘
                             │
              ┌──────────────▼──────────────────┐
              │ User Query (RAG agent)          │
              ├──────────────┬──────────────────┤
              │ Embed query  │ Retrieve Top-K   │
              │ Re-rank      │ Apply Guardrails │
              └──────────────┴──────────────────┘
                             │
                 ┌───────────▼────────────┐
                 │ Final LLM Response     │
                 └────────────────────────┘


┌────────────────────────────────────────┐
│        SharePoint Online (SPO)         │
│     - Document Libraries (Millions)    │
└──────────────────────────┬─────────────┘
                           │
      ┌────────────────────▼────────────────────┐
      │        Microsoft Graph API              │
      │   - @odata.deltaLink delta query for    |
      |          incremental sync               |
      │   - webhook notifications for triggers  │
      └──────────────┬───────────────┬──────────┘
                     │               │
     (push) Webhook  │               │  (pull) Scheduled job
                     ▼               ▼
         ┌────────────────┐    ┌────────────────────┐
         │ Azure Function │    │ Databricks Job     │
         │ (Webhook       │    │                    │
         │  Listener)     │    └──────────┬─────────┘
         └────────────────┘               ▼
                                  ┌──────────────┐
                                  │ DeltaLink DB │
                                  │ (Azure SQL)  │
                                  └─────┬────────┘
                                        ▼
                             ┌─────────────────────┐
                             │ Graph API (delta)   │
                             │ - Get new/updated   │
                             │ - Use deltaLink     │
                             └────────┬────────────┘
                                      ▼
                         ┌────────────────────────────┐
                         │ Azure Data Lake Storage    │
                         │ (Raw Zone: PDF/DOCX/etc.)  │
                         └────────────┬───────────────┘
                                      ▼
                     ┌─────────────────────────────────┐
                     │ Databricks Spark Pipeline       │
                     │ - Extract text (Document Intel) │
                     │ - PII redaction (Spark NLP)     │
                     │ - LLM classification (GPT-4)    │
                     └────────────┬────────────────────┘
                                  ▼
                     ┌─────────────────────────────┐
                     │ Human-in-the-loop UI        │
                     │ - Accept / correct labels   |
                     |     Tagging                 │
                     └────────────┬────────────────┘
                                  ▼
                     ┌─────────────────────────────┐
                     │ Azure OpenAI Embeddings     │
                     │ - Generate vector for       |
                     |      multimodal data        │
                     └────────────┬────────────────┘
                                  ▼
                     ┌─────────────────────────────┐
                     │ Azure Cognitive Search (ACS)│
                     │ - Store vectors + metadata  │
                     └────────────┬────────────────┘
                                  ▼
                    ┌────────────────────────────────────┐
                    │ Agent (Azure OpenAI + Guardrails)│
                    │ - Embed user query                  │
                    │ - Retrieve docs via ACS             │
                    │ - Re-rank + Filter (guardrails)     │
                    │ - Final GPT-4 Answer                │
                    └────────────────────────────────────┘
